This tutorial is modified from: https://sep.readthedocs.io/en/v1.0.x/tutorial.html

Reminder - comment out pngs if not currently looking to save!!!

12/2/21: Fenix cloned tutorial code and changed file name to "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits" 

In [ ]:
import numpy as np
import sep

In [ ]:
# additional setup for reading the test image and displaying plots
import astropy
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
hdu_list = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
hdu_list.info()
print(hdu_list[0].header)


image_data = hdu_list[0].data

#necessary to reorder the array when using SEP!!!
image_data = image_data.byteswap().newbyteorder()

print(image_data.shape)

# show the image
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

#Saving the original image plot as a png
#plt.savefig('tutorial_img_1.png',bbox_inches="tight",dpi=600)

#looking for zeropoint in the wide field camera 3
#zero point for wfc3 in 3012 - 26.2687
#https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/ir-photometric-calibration

In [ ]:
#different color for funsies
plt.imshow(image_data, interpolation='nearest', cmap='cividis', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# Background subtraction

In [ ]:
# measure a spatially varying background on the image

bkg = sep.Background(image_data)


In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
#len(bkg_image)
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

#Saving the background plot as a png
#plt.savefig('tutorial_img_2.png',bbox_inches="tight",dpi=600)

## Mark down/comments for cells 9-18 - Fenix

The .rms member function the SEP background object (denoted as 'bkg' and initialized in cell 5) evaluates on the background as a 2d array and computes the root-mean-squared, or standard deviation, for each element(or pixel). It then creates a new array (equal in size to bkg) containing these calculations. 

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

#Printing to confirm image and noise are in same sized arrays
print("Length of bkg_image: ",len(bkg_image))
print("Length of bkg_rms: ",len(bkg_rms))

This cell plots the previously calculated standard deviation, or optical noise, of the background, pixel by pixel. This noise is normally attributed to the electronic noise of physical instrumentation/cameras used and the surrounding light pollution from when the image was taken.

In [ ]:
# show the background noise

#interpolation='nearest' makes sure that the pixels are plotted individually and 
#not artificially adjusted to increase the resolution of the display
#cmap='gray' colors the plot as a grayscale color scheme
#origin='lower' tells the plotting of data to begin from the bottom left
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')

#this plots the color bar alongside the noise plot
#based on the color scheme chosen, rms values below 62 are shown as black
#rms values above 68 are white
#rms values between 62-68 are on a grayscale that lightens as the values increase
plt.colorbar();

#Saving the background noise plot as a png
#plt.savefig('tutorial_img_3.png',bbox_inches="tight",dpi=600)

This cell removes the background value from each pixel in the original image. This makes it easier in the future to determine a threshold value that will determine what is considered an object and what was simply empty space in the image. This can be thought of an an array being subtracted from another of equal size. 

In [ ]:
# subtract the background
data_sub = image_data - bkg

# Object detection

The extract function uses numerous parameters to detect objects within an image. The general form (found on https://sep.readthedocs.io/en/v1.0.x/api/sep.extract.html) is: 

sep.extract(data, thresh, err=None, mask=None, minarea=5, filter_kernel=default_kernel, filter_type='matched', deblend_nthresh=32, deblend_cont=0.005, clean=True, clean_param=1.0, segmentation_map=False)

In [ ]:
#Matching parameters to the general form:
    #data_sub is the array of the background-subtracted image. This is the array
        #that "extract" will operate on
    #1.5 is the float threshold value we will multiply by the error.
    #err=bkg.globalrms is a float and the "global" noise of the image background
    #Several more parameters can be added to create a more specialized search.
    
#The function expect will compare each element in data_sub to 1.5err. A pixel above the 
    #threshold is considered to be part of an object
#The objects detected will be returned as an array
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

This cell confirms that the number of objects detected is equal to the length of the array returned by "extract" function.

In [ ]:
# how many objects were detected
print("Number of objects detected:",len(objects))

Now that the objects have been detected within the image, they can be visually highlighted with ellipses. This makes it easier to locate where these objects are on the image itself.

First, we need to import the ellipse-shaped patch to be superimposed over the image. 

Note that we plot the background-subtracted image as opposed to the original. This is because the background-subtracted version was used to find the objects. 

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image

#we use subplots because we are going to place a plot of ellipses
#over the image plot so that they overlap. Together, this indicates
#where the objects are visually. 
fig, ax = plt.subplots()

#m is the average value of the data_sub array
#s is the standard deviation of the data set from the average
m, s = np.mean(data_sub), np.std(data_sub)

#this plots the data_sub image first
    #interpolation='nearest' makes sure that the pixels are plotted individually and 
        #not artificially adjusted to increase the resolution of the display
    #cmap='gray' colors the plot as a grayscale color scheme
    #vmin is the minimum variability
    #vmax is the maximum variability
    #origin='lower' tells the plotting of data to begin from the bottom left
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
#i will iterate from 0 to 67 since len(objects)=68
for i in range(len(objects)):
    #xy are the coordinates of the ellipse center, which are given by the
        #ith terms of the x and y centroid coordinates
    #the width is the horizontal diameter, scaled by 6
    #the height is the vertical diameter, scaled by 6
    #the angle rotates the ellipse by 180/pi
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    
    e.set_facecolor('none') #this makes the ellipse an empty outline without shading inside
    e.set_edgecolor('red')  #this makes the ellipse outline red
    
    #this updates the plot with the newly oriented ellipse to indicate the ith object 
    #on the data-subtracted image
    #ellipses are added starting at the bottom left
    ax.add_artist(e)

#Saving the object search plot as a png
#plt.savefig('tutorial_img_4.png',bbox_inches="tight",dpi=600)


This cell serves to show all the additional data analysis functions that can be used on the array object (contains number of objects detected in image and where they are located). For example, we can find the minimum and maximum coordinates of pixels within the object array using (xmin,ymin) and (xmax,ymax). Further information on these parameters can be found at the bottom of the page here: https://sep.readthedocs.io/en/v1.0.x/api/sep.extract.html

In [ ]:
# available fields
objects.dtype.names

# Aperture photometry

Aperture photometry finds the fluctuation between pixels in a circle within a certain radius and the surrounding pixels just outside of that circle. The following cell finds the fluctuation between pixels for the previously detected objects.

The function sum_circle sums up the pixels within the given circle, calculates the error, and gives an array of flags.The general form of the sum_circle function can be found on https://sep.readthedocs.io/en/v1.0.x/api/sep.sum_circle.html and is as follows: 

sep.sum_circle(data, x, y, r, err=None, var=None, mask=None, maskthresh=0.0, bkgann=None, gain=None, subpix=5)

In [ ]:
#flux, fluxerr, and flag will all be arrays
#data_sub is the array to be operated on
#object['x'] is the x centroid coordinate of a given circle
#object['y'] is the y centroid coordinate of a given circle
#2.0 is radius, aka 2 pixels (changed from previous 3 to remove negative fluxes)
#err=bkg.globalrms sets the error to be the constant noise (a float)
#gain=1.0 accounts for poisson noise in aperture sum
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     2.0, err=bkg.globalrms, gain=1.0)

This cells prints out the first 10 objects results (out of 68) calculated in the previous cell by sum_circle. 

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

# Histogram the fluxes

In [ ]:
#from astropy, plot histogram with matplot, cast data from a 2d array
#zoomed in on the range [-0.25,0.25]
histogram = plt.hist(flux.flatten(),bins='auto',range=[-0.25,0.25])

## Converting flux to AB magnitude - Fenix and Neil
Discussion section 12/2/21 3:20-4:25pm

In [ ]:
#looking for zeropoint in the wide field camera 3
#zero point for wfc3 in 3012 - 26.2687
ABMag = -2.5*np.log10(flux) - 26.2687
for i in range(10):
    print("object {:d}: flux = {:f}".format(i, ABMag[i]))


In [ ]:
histogram = plt.hist(ABMag.flatten(),bins='auto',range=[-30,-20])